The idea here is to look at the evolution of a single stegoton as it moves through one period of the medium.  We can compare its behavior with what is predicted by certain intermediate equations obtained in the homogenization process.

Before executing this notebook, it's necessary to run `stegoton.py` with appropriate parameters.

In [ ]:
from clawpack import pyclaw
import matplotlib.pyplot as plt
import numpy as np
from stegoton import cells_per_layer


# Cutting the stegotons

In [ ]:
soln = pyclaw.Solution()

soln.read(20,file_format='ascii')
xc = soln.grid.x.centers
plt.plot(-soln.q[1,:])


In [ ]:
q0 = soln.q.copy()

isteg1 = [2000,3000]  # This depends on the setup in stegoton.py
plt.plot(q0[0,isteg1[0]:isteg1[1]])  # Plot to check that we got the right region


In [ ]:
steg1_shift = -30  # Move the stegoton this far, in material layers
q = np.zeros_like(q0)
isteg1_shifted = np.array(isteg1) + cells_per_layer*steg1_shift
q[:,isteg1_shifted[0]:isteg1_shifted[1]] = q0[:,isteg1[0]:isteg1[1]]
plt.plot(xc,q[0,:])

In [ ]:
np.savetxt('shorter_stegoton.txt',q)


In [ ]:
from stegoton import setup
claw = setup(ic='shorter_stegoton.txt',tfinal=10,outdir='shorter')
claw.run()


In [ ]:
sol_1steg = pyclaw.Solution()

sol_1steg.read(2000,path='./shorter',file_format='ascii',read_aux=True)
xc = sol_1steg.grid.x.centers
plt.plot(xc,sol_1steg.q[0,:])


# Plotting

In [ ]:
%matplotlib inline
import matplotlib
from matplotlib import animation
matplotlib.rcParams['animation.embed_limit'] = 2**128
from IPython.display import HTML

fig, ax = plt.subplots(figsize=[12,8], nrows=2, ncols=1, sharex=True)
fig.subplots_adjust(hspace=0)

#fig = plt.figure(figsize=[10,6])
#ax = plt.axes(xlim=(xc[0], xc[-1]), ylim=(2.4, 2.9))
ax[0].set_ylim(-0.02,0.25)
ax[1].set_ylim(-0.02,0.4)
line, = ax[0].plot([], [], lw=2)
line2, = ax[1].plot([], [], lw=2)

def fplot(i):
    t = i*claw.tfinal/claw.num_output_times
    #tx = 1.21*t + 20
    xl = 23
    xr = 43

    frame = claw.frames[i]
    eps = frame.q[0,:]
    rho = frame.aux[0,:]
    K = frame.aux[1,:]
    sigma = np.exp(K*eps)-1
    q2 = frame.q[1,:]
    line.set_data(xc, -q2/rho)
    line2.set_data(xc, sigma)
    ax[0].set_title("t= {:.1f}".format(frame.t))
    ax[0].set_ylabel('-u',fontsize=20)
    ax[1].set_ylabel('Stress',fontsize=20)
    ax[0].set_xlim(xl,xr)
    ax[1].set_xlim(xl,xr)
    
    fname = 'frame'+str(i).zfill(4)+'.png'
    fig.savefig(fname)
    
    return line,

anim = animation.FuncAnimation(fig, fplot, frames=1000, repeat=False)
plt.close()
HTML(anim.to_jshtml())

In [ ]:
# Write video to file
writervideo = animation.FFMpegWriter(fps=20)
anim.save('breather_shorter.mp4',writervideo)


# Speed

In [ ]:
x0 = xc[np.argmin(claw.frames[0].q[1,:])]
t0 = claw.frames[0].t

s = np.zeros(245)

for i, end in enumerate(range(1,len(s)+1)):
    x1 = xc[np.argmin(claw.frames[-end].q[1,:])]
    t1 = claw.frames[-end].t
    s[i] = (x1-x0)/(t1-t0)

s_mean = np.sum(s)/len(s)
plt.plot(s)
s[0]-s[-1]
#s_mean

In [ ]:
s_mean

Now we should re-run with output at intervals `s_mean*dx`

Next let's look at the solution divided by the initial data, in the moving frame of reference.

In [ ]:
init = claw.frames[0]
eps0 = init.q[0,:]
K = init.aux[1,:]
sigma0 = np.exp(K*eps0) - 1

fig, ax = plt.subplots(figsize=[12,8], nrows=2, ncols=1, sharex=True)
fig.subplots_adjust(hspace=0)

#fig = plt.figure(figsize=[10,6])
#ax = plt.axes(xlim=(xc[0], xc[-1]), ylim=(2.4, 2.9))
ax[0].set_ylim(-0.02,0.25)
ax[1].set_ylim(-0.02,0.4)
line, = ax[0].plot([], [], lw=2)
line2, = ax[1].plot([], [], lw=2)

def fplot(i):
    t = i*claw.tfinal/claw.num_output_times
    #tx = 1.21*t + 20
    xl = 23
    xr = 43

    frame = claw.frames[i]
    eps = frame.q[0,:]
    rho = frame.aux[0,:]
    K = frame.aux[1,:]
    sigma = np.exp(K*eps)-1
    q2 = frame.q[1,:]
    line.set_data(xc, -q2/rho)
    line2.set_data(xc, sigma)
    ax[0].set_title("t= {:.1f}".format(frame.t))
    ax[0].set_ylabel('-u',fontsize=20)
    ax[1].set_ylabel('Stress',fontsize=20)
    ax[0].set_xlim(xl,xr)
    ax[1].set_xlim(xl,xr)
    
    fname = 'frame'+str(i).zfill(4)+'.png'
    fig.savefig(fname)
    
    return line,

anim = animation.FuncAnimation(fig, fplot, frames=1000, repeat=False)
plt.close()
HTML(anim.to_jshtml())

# Averaged solution

In [ ]:
num_layers = len(xc)//cells_per_layer
qq = claw.frames[0].q
for i in range(num_layers):
    qq[0,i*cells_per_layer:(i+1)*cells_per_layer] = sum(qq[0,i*cells_per_layer:(i+1)*cells_per_layer])/cells_per_layer 
    qq[1,i*cells_per_layer:(i+1)*cells_per_layer] = sum(qq[1,i*cells_per_layer:(i+1)*cells_per_layer])/cells_per_layer
    
plt.plot(xc,qq[0,:])
plt.xlim(23,43)
    
    

In [ ]:
# Plot with averaged solution

%matplotlib inline
import matplotlib
from matplotlib import animation
matplotlib.rcParams['animation.embed_limit'] = 2**128
from IPython.display import HTML

fig, ax = plt.subplots(figsize=[12,8], nrows=2, ncols=1, sharex=True)
fig.subplots_adjust(hspace=0)

#fig = plt.figure(figsize=[10,6])
#ax = plt.axes(xlim=(xc[0], xc[-1]), ylim=(2.4, 2.9))
ax[0].set_ylim(-0.02,0.75)
ax[1].set_ylim(-0.02,1.0)
line, = ax[0].plot([], [], lw=2)
lineavg, = ax[1].plot([], [], '-o', lw=2)
line2, = ax[1].plot([], [], lw=2)

def fplot(i):
    t = i*claw.tfinal/claw.num_output_times
    #tx = 1.21*t + 20
    xl = 23
    xr = 43

    frame = claw.frames[i+1]
    eps = frame.q[0,:]
    K = frame.aux[1,:]
    sigma = np.exp(K*eps)-1
    u = frame.q[1,:]
    
    # Compute averages of sigma
    sig_avg = np.zeros(num_layers)
    for i in range(num_layers):
        sig_avg[i] = sum(sigma[i*cells_per_layer:(i+1)*cells_per_layer])/cells_per_layer     
    
    line.set_data(xc, eps)
    lineavg.set_data(np.arange(num_layers)+1/2,sig_avg)
    line2.set_data(xc, sigma)
    ax[0].set_title("t= {:.1f}".format(frame.t))
    ax[0].set_ylabel('Strain',fontsize=20)
    ax[1].set_ylabel('Stress',fontsize=20)
    ax[0].set_xlim(xl,xr)
    ax[1].set_xlim(xl,xr)
    
    fname = 'frame'+str(i).zfill(4)+'.png'
    fig.savefig(fname)
    
    return line,

anim = animation.FuncAnimation(fig, fplot, frames=1000, repeat=False)
plt.close()
HTML(anim.to_jshtml())